# Riemann-ML Pipeline: Run Simulation

Este notebook reproduz o pipeline de execução do solucionador FVM (esquema de Rusanov) aplicado ao tubo de choque de Sod. O objetivo é gerar os perfis finais de densidade, velocidade e pressão e armazenar os artefatos correspondentes.

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
SRC_PATH = PROJECT_ROOT / 'src'
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))


In [ ]:
from pathlib import Path

import numpy as np

from IPython.display import Image, display

from riemann_ml.core.euler1d import StatePrim
from riemann_ml.fvm.solver import simulate
from riemann_ml.utils.plotting import plot_profiles

In [ ]:
# Parâmetros principais do experimento
GAMMA = 1.4
NUM_CELLS = 400
CFL = 0.5
FINAL_TIME = 0.2
INTERFACE_POSITION = 0.5
ARTIFACT_DIR = Path("data/artifacts/fvm_sod_notebook")
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Estados primitivos do problema de Sod
left_state = StatePrim(density=1.0, velocity=0.0, pressure=1.0)
right_state = StatePrim(density=0.125, velocity=0.0, pressure=0.1)

In [ ]:
# Executa a simulação FVM com esquema de Rusanov
times, x, conservative_state, _ = simulate(
    num_cells=NUM_CELLS,
    final_time=FINAL_TIME,
    cfl=CFL,
    left_state=left_state,
    right_state=right_state,
    gamma=GAMMA,
    interface_position=INTERFACE_POSITION,
    store_history=False,
)

print(f"Final time: {times[-1]:.6f} s | Total steps: {len(times) - 1}")

In [ ]:
# Converte o estado conservativo para variáveis primitivas para análise
rho = conservative_state[:, 0]
momentum = conservative_state[:, 1]
energy = conservative_state[:, 2]
velocity = momentum / np.clip(rho, 1e-12, None)
pressure = (GAMMA - 1.0) * np.maximum(energy - 0.5 * momentum**2 / np.clip(rho, 1e-12, None), 1e-12)

solution_path = ARTIFACT_DIR / "solution_sod.npz"
np.savez(
    solution_path,
    times=times,
    x=x,
    q=conservative_state,
    gamma=GAMMA,
)
print(f"Persisted simulation state to {solution_path}")

In [ ]:
# Gera e exibe os perfis de densidade, velocidade e pressão
plot_path = ARTIFACT_DIR / f"profiles_t{times[-1]:.3f}.png"
plot_profiles(
    x=x,
    density=rho,
    velocity=velocity,
    pressure=pressure,
    title=f"Sod shock tube | t = {times[-1]:.3f} s",
    savepath=plot_path,
)

print(f"Saved plot to {plot_path}")
display(Image(filename=str(plot_path)))